In [22]:
import requests
import json
import html2text
import bs4

In [ ]:
# https://discourse.onlinedegree.iitm.ac.in/search?q=%23courses%3Atds-kb%20before%3A2025-04-14&page=1
# https://discourse.onlinedegree.iitm.ac.in/search?q=%23courses%3Atds-kb%20before%3A2025-04-14&page=2
# https://discourse.onlinedegree.iitm.ac.in/search?q=%23courses%3Atds-kb%20before%3A2025-04-14&page=3

In [10]:
with open('cookie.txt', 'r') as f:
    cookie = f.read().strip()
url = "https://discourse.onlinedegree.iitm.ac.in/search?q=%23courses%3Atds-kb%20before%3A2025-04-14&page=1"
headers = {
    "cookie": cookie
}

In [11]:
response = requests.get(url, headers=headers)

In [13]:
response

<Response [200]>

In [17]:
print(html2text.html2text(response.text))

[IITM-DSA](/)

![IITM-DSA](https://europe1.discourse-
cdn.com/flex013/uploads/iitm/original/3X/1/8/18057f852cd1b347f4f0a09449d49632e5fb7d45.svg)

# Welcome to IITM-DSA

A place for IITM BS Degree students to build and share knowledge

  * [Home ](/)
  * [Categories ](/categories)
  * [Guidelines ](/guidelines)
  * [Terms of Service ](/tos)
  * [Privacy Policy ](/privacy)

Powered by [Discourse](https://www.discourse.org), best viewed with JavaScript
enabled




In [19]:
# print(response.text)

In [21]:
response

<Response [200]>

In [24]:
import requests
import os
import json
from datetime import datetime, timezone # Ensure timezone is imported
from urllib.parse import urljoin, urlencode

In [23]:
DISCOURSE_BASE_URL = "https://discourse.onlinedegree.iitm.ac.in/"
CATEGORY_SLUG = "courses/tds-kb"
CATEGORY_ID = 34
START_DATE = "2025-01-01" # Inclusive
END_DATE = "2025-04-15"   # Inclusive"

In [25]:
RAW_COOKIE_STRING = """""" # Replace with your actual cookie string

OUTPUT_DIR = "discourse_json"
POST_ID_BATCH_SIZE = 50
MAX_CONSECUTIVE_PAGES_WITHOUT_NEW_TOPICS = 5 # New configuration for breaking loop

In [26]:
# urljoin(base_url, f"c/{category_slug}/{category_id}.json")
url = "https://discourse.onlinedegree.iitm.ac.in/c/courses/tds-kb/34"
topic_ids = []

In [28]:
datetime.fromisoformat(START_DATE + "T00:00:00").replace(tzinfo=timezone.utc)

datetime.datetime(2025, 1, 1, 0, 0, tzinfo=datetime.timezone.utc)

In [29]:
datetime.fromisoformat(END_DATE + "T23:59:59.999999").replace(tzinfo=timezone.utc)

datetime.datetime(2025, 4, 15, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)

In [30]:
paginated_url = f"{url}?page={1}"
requests.get(paginated_url, cookies=cookie, timeout=30)

TypeError: string indices must be integers, not 'str'

In [1]:
import time
from semantic_text_splitter import MarkdownSplitter
from google import genai
from config import GEMINI_API_KEY
from pathlib import Path
from tqdm import tqdm
import numpy as np

class RateLimiter:
    def __init__(self, requests_per_minute: int = 60, requests_per_second: int = 2):
        self.requests_per_minute = requests_per_minute
        self.requests_per_second = requests_per_second
        self.request_times = []
        self.last_request_time = 0
        
    
    def wait(self):
        curr_time = time.time()
        
        # per-second rate limiting
        time_since_last_request = curr_time - self.last_request_time
        if time_since_last_request < (1 / self.requests_per_second):
            time.sleep((1 / self.requests_per_second) - time_since_last_request)
        
        # per-minute rate limiting
        curr_time = time.time()
        self.request_times = [t for t in self.request_times if t > curr_time - 60]
        if len(self.request_times) >= self.requests_per_minute:
            sleep_time = 60 - (curr_time - self.request_times[0])
            if sleep_time > 0:
                time.sleep(sleep_time)
                curr_time = time.time()
                self.request_times = [t for t in self.request_times if t > curr_time - 60]
                
        
        self.request_times.append(curr_time)
        self.last_request_time = curr_time
        
rate_limiter = RateLimiter(requests_per_minute=5, requests_per_second=2)

def get_embeddings(text: str, model: str = "gemini-embedding-exp-03-07", max_tries: int = 3):
    client = genai.Client(api_key=GEMINI_API_KEY)

    for attempt in range(max_tries):
        try:
            rate_limiter.wait()
            response = client.models.embed_content(
                model=model,
                contents=text
            )
            
            
            return response.embeddings[0].values
        except Exception as e:
            if "rate limit" in str(e).lower() or "quota" in str(e).lower():
                # exponential backoff for rate limit errors
                wait_time = 2 ** attempt
                print(f"Rate limit exceeded, retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            elif attempt == max_tries - 1:
                print(f"Failed to get embeddings after {max_tries} attempts: {e}")
                raise
            else:
                print(f'attempt {attempt + 1} failed with error: {e}, retrying...')
                time.sleep(1)  # wait before retrying

def get_chunks(file_path: str, chunk_size: int = 1000):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    splitter = MarkdownSplitter(chunk_size)
    chunks = splitter.chunks(text)
    return chunks

if __name__ == "__main__":
    files = [*Path("raw-data/Markdown-data").glob("*.md")]
    all_chunks = []
    all_embeddings = []
    total_chunks = 0
    file_chunks = {}
    for file_path in files:
        chunks = get_chunks(file_path)
        file_chunks[file_path] = chunks
        total_chunks += len(chunks)
    print(f"Total chunks created: {total_chunks}")
    
    with tqdm(total=total_chunks, desc="Processing Chunks") as pbar:
        for file_path, chunks in file_chunks.items():
            try:
                for chunk in chunks:
                    embedding = get_embeddings(chunk)
                    all_chunks.append(chunk)
                    all_embeddings.append(embedding)
                    pbar.set_postfix({"file": file_path.name, "chunk": len(all_chunks)})
                    pbar.update(1)
            except Exception as e:
                print(f"Error processing {file_path}: {e}")
                pbar.update(1)
                continue
    
    # save file to .npz
    np.savez("embeddings.npz", chunks=all_chunks, embeddings=all_embeddings)


Total chunks created: 541


Processing Chunks:   0%|          | 1/541 [00:06<55:36,  6.18s/it, file=1._Development_Tools.md, chunk=1]


KeyboardInterrupt: 

In [3]:
print(all_chunks)

['---\ntitle: "1. Development Tools"\noriginal_url: "https://tds.s-anand.net/#/development-tools?id=development-tools"\ndownloaded_at: "2025-05-31T21:34:43.013026"\n---\n\n[Development Tools](#/development-tools?id=development-tools)\n=============================================================\n\n**NOTE**: The tools in this module are **PRE-REQUISITES** for the course. You would have used most of these before. If most of this is new to you, please take this course later.\n\nSome tools are fundamental to data science because they are industry standards and widely used by data science professionals. Mastering these tools will align you with current best practices and making you more adaptable in a fast-evolving industry.\n\nThe tools we cover here are not just popular, they’re the core technology behind most of today’s data science and software development.\n\n[Previous\n\nTools in Data Science](#/README)\n\n[Next\n\nEditor: VS Code](#/vscode)']


In [4]:
all_embeddings

[[0.0007703485,
  0.00016222592,
  -0.0034708588,
  -0.050694283,
  0.00843989,
  0.04154368,
  0.01471481,
  -0.014705877,
  -0.00013857722,
  0.0016048915,
  -0.024009671,
  -0.019763002,
  0.03294355,
  0.031457305,
  0.14680986,
  -0.0025977343,
  0.0141352285,
  -0.018821038,
  0.010907307,
  -0.022563694,
  0.007292424,
  0.021419706,
  -0.009889426,
  -0.010477611,
  0.017178327,
  -0.015657533,
  0.012438986,
  -0.0047027674,
  0.064140014,
  0.009123775,
  -0.010807903,
  0.0037541543,
  -0.0036704438,
  0.054082952,
  0.020960424,
  0.035937745,
  0.007216362,
  -0.006239389,
  0.02705876,
  0.00013644177,
  -0.012683685,
  -0.0016873349,
  0.011119196,
  -0.036123272,
  -0.00476877,
  0.030429156,
  0.009734907,
  0.0034182766,
  -0.011180014,
  0.0251792,
  0.0017435241,
  0.007913748,
  -0.01874881,
  -0.17818744,
  -0.012355322,
  -0.008221409,
  -0.009183192,
  0.012737399,
  -0.02448687,
  0.0022497429,
  -0.023741461,
  0.005860368,
  -0.048816785,
  0.00284152,
  0.00

In [5]:
file_chunks

{WindowsPath('raw-data/Markdown-data/1._Development_Tools.md'): ['---\ntitle: "1. Development Tools"\noriginal_url: "https://tds.s-anand.net/#/development-tools?id=development-tools"\ndownloaded_at: "2025-05-31T21:34:43.013026"\n---\n\n[Development Tools](#/development-tools?id=development-tools)\n=============================================================\n\n**NOTE**: The tools in this module are **PRE-REQUISITES** for the course. You would have used most of these before. If most of this is new to you, please take this course later.\n\nSome tools are fundamental to data science because they are industry standards and widely used by data science professionals. Mastering these tools will align you with current best practices and making you more adaptable in a fast-evolving industry.\n\nThe tools we cover here are not just popular, they’re the core technology behind most of today’s data science and software development.\n\n[Previous\n\nTools in Data Science](#/README)\n\n[Next\n\nEdito

In [7]:
data = np.load("embeddings.npz", allow_pickle=True)

In [10]:
data['embeddings']

array([[ 0.00077035,  0.00016223, -0.00347086, ..., -0.0017963 ,
         0.00942388,  0.01848597],
       [-0.0005184 , -0.01162009,  0.00195142, ...,  0.00918271,
         0.02848052, -0.00432788],
       [ 0.00895323,  0.00929906,  0.02771124, ..., -0.00870913,
         0.0160321 ,  0.00478223],
       ...,
       [-0.01058022,  0.0032924 ,  0.01653316, ..., -0.00936417,
         0.00340856, -0.01460751],
       [-0.00951528, -0.0114082 ,  0.02206926, ..., -0.01311319,
        -0.00230006, -0.00397046],
       [ 0.01077425, -0.00350613,  0.01643834, ..., -0.00349778,
         0.00308677, -0.00856224]], shape=(10, 3072))

In [11]:
data['chunks'].tolist()

['---\ntitle: "1. Development Tools"\noriginal_url: "https://tds.s-anand.net/#/development-tools?id=development-tools"\ndownloaded_at: "2025-05-31T21:34:43.013026"\n---\n\n[Development Tools](#/development-tools?id=development-tools)\n=============================================================\n\n**NOTE**: The tools in this module are **PRE-REQUISITES** for the course. You would have used most of these before. If most of this is new to you, please take this course later.\n\nSome tools are fundamental to data science because they are industry standards and widely used by data science professionals. Mastering these tools will align you with current best practices and making you more adaptable in a fast-evolving industry.\n\nThe tools we cover here are not just popular, they’re the core technology behind most of today’s data science and software development.\n\n[Previous\n\nTools in Data Science](#/README)\n\n[Next\n\nEditor: VS Code](#/vscode)',
 '---\ntitle: "2. Deployment Tools"\nori

In [3]:
from semantic_text_splitter import MarkdownSplitter

In [1]:
with open(r'raw-data\Markdown-data\2._Deployment_Tools.md', 'r', encoding='utf-8') as file:
    text = file.read()
    splitter = MarkdownSplitter(1000)
    chunks = splitter.chunks(text)

NameError: name 'MarkdownSplitter' is not defined

In [5]:
text

'---\ntitle: "2. Deployment Tools"\noriginal_url: "https://tds.s-anand.net/#/deployment-tools?id=deployment-tools"\ndownloaded_at: "2025-05-31T21:34:11.680843"\n---\n\n[Deployment Tools](#/deployment-tools?id=deployment-tools)\n==========================================================\n\nAny application you build is likely to be deployed somewhere. This section covers the most popular tools involved in deploying an application.\n\n[Previous\n\nVersion Control: Git, GitHub](#/git)\n\n[Next\n\nMarkdown](#/markdown)'

In [9]:
chunks

['---\ntitle: "2. Deployment Tools"\noriginal_url: "https://tds.s-anand.net/#/deployment-tools?id=deployment-tools"\ndownloaded_at: "2025-05-31T21:34:11.680843"\n---\n\n[Deployment Tools](#/deployment-tools?id=deployment-tools)\n==========================================================\n\nAny application you build is likely to be deployed somewhere. This section covers the most popular tools involved in deploying an application.\n\n[Previous\n\nVersion Control: Git, GitHub](#/git)\n\n[Next\n\nMarkdown](#/markdown)']

In [11]:
import frontmatter

def preprocess_markdown(file_path):
    # Load the markdown file and parse frontmatter
    post = frontmatter.load(file_path)
    
    # Extract the original_url from the frontmatter
    # original_url = post.get('original_url', '')
    
    # Get the main content (without frontmatter)
    content = post.content.strip()
    
    # Combine original_url and main content
    preprocessed_text = f"{content}"
    
    return preprocessed_text

In [12]:
with open(r'raw-data\Markdown-data\2._Deployment_Tools.md', 'r', encoding='utf-8') as file:
    text = file.read()
    # splitter = MarkdownSplitter(1000)
    # chunks = splitter.chunks(text)

In [28]:
print(preprocess_markdown(r'raw-data\Markdown-data\Wikipedia_Data_with_Python.md'))

Wikipedia Data with Python




You’ll learn how to scrape data from Wikipedia using the 
wikipedia
 Python library, covering:




Installing and Importing
: Use pip install to get the Wikipedia library and import it with import wikipedia as wk.


Keyword Search
: Use the search function to find Wikipedia pages containing a specific keyword, limiting results with the results argument.


Fetching Summaries
: Use the summary function to get a concise summary of a Wikipedia page, limiting sentences with the sentences argument.


Retrieving Full Pages
: Use the page function to obtain the full content of a Wikipedia page, including sections and references.


Accessing URLs
: Retrieve the URL of a Wikipedia page using the url attribute of the page object.


Extracting References
: Use the references attribute to get all reference links from a Wikipedia page.


Fetching Images
: Access all images on a Wikipedia page via the images attribute, which returns a list of image URLs.


Extracting Ta

In [20]:
import frontmatter
import re
from markdown import markdown
from bs4 import BeautifulSoup

def preprocess_markdown(filepath):
    # Load markdown file and extract frontmatter
    post = frontmatter.load(filepath)
    # title = post.get('title', '')
    original_url = post.get('original_url', '')
    # Remove downloaded_at if present (not needed)
    
    # Get markdown content
    content = post.content

    # Remove navigation links (e.g., [Previous ...], [Next ...])
    # This regex removes lines starting with [Previous or [Next (case-insensitive)
    content = re.sub(r'^\[(Previous|Next)[\s\S]*?\]\([^\)]*\)\s*', '', content, flags=re.MULTILINE|re.IGNORECASE)
    
    # Convert markdown to HTML, then HTML to plain text
    html = markdown(content)
    soup = BeautifulSoup(html, "html.parser")
    plain_text = soup.get_text(separator='\n')

    # Assemble final processed text
    processed = f"{plain_text.strip()}\n\nSource: {original_url}"
    return processed


In [22]:
# processed_text = preprocess_markdown(r"raw-data\Markdown-data\Scraping_with_Google_Sheets.md")
# print(processed_text)

In [11]:
from pathlib import Path
import numpy as np
from tqdm import tqdm
from embed import get_chunks, get_embeddings
from config import GEMINI_API_KEY
from extract_text import extract_text_from_markdown


def process_save_markdown():
    count = 0
    files = [*Path("raw-data/Markdown-data").glob("*.md")]
    all_chunks = []
    urls = []
    all_embeddings = []
    total_chunks = 0
    file_chunks = {}
    for file_path in files:
        content, url = extract_text_from_markdown(file_path)
        chunks = get_chunks(content)
        file_chunks[file_path] = chunks
        total_chunks += len(chunks)
        print(f"File: {file_path.name}, Chunks: {chunks} \n \n")
        count += 1
        if count == 10:
            break

In [12]:
process_save_markdown()

File: 1._Development_Tools.md, Chunks: ['Development Tools\n\n\nNOTE\n: The tools in this module are \nPRE-REQUISITES\n for the course. You would have used most of these before. If most of this is new to you, please take this course later.\n\n\nSome tools are fundamental to data science because they are industry standards and widely used by data science professionals. Mastering these tools will align you with current best practices and making you more adaptable in a fast-evolving industry.\n\n\nThe tools we cover here are not just popular, they’re the core technology behind most of today’s data science and software development.'] 
 

File: 2._Deployment_Tools.md, Chunks: ['Deployment Tools\n\n\nAny application you build is likely to be deployed somewhere. This section covers the most popular tools involved in deploying an application.'] 
 

File: 3._Large_Language_Models.md, Chunks: ['Large Language Models\n\n\nThis module covers the practical usage of large language models (LLMs).\n\n

In [2]:
import frontmatter
import re
from markdown import markdown
from bs4 import BeautifulSoup

def extract_text_from_markdown(filepath):
    # Load markdown file and extract frontmatter
    post = frontmatter.load(filepath)
    # title = post.get('title', '')
    original_url = post.get('original_url', '')
    # Remove downloaded_at if present (not needed)
    
    # Get markdown content
    content = post.content

    # Remove navigation links (e.g., [Previous ...], [Next ...])
    # This regex removes lines starting with [Previous or [Next (case-insensitive)
    content = re.sub(r'^\[(Previous|Next)[\s\S]*?\]\([^\)]*\)\s*', '', content, flags=re.MULTILINE|re.IGNORECASE)
    
    # Convert markdown to HTML, then HTML to plain text
    html = markdown(content)
    soup = BeautifulSoup(html, "html.parser")
    plain_text = soup.get_text(separator='\n')

    return plain_text.strip(), original_url


In [5]:
from pathlib import Path
import numpy as np
from tqdm import tqdm
from embed import get_chunks, get_embeddings
from config import GEMINI_API_KEY
all_chunks = []
all_embeddings = []
all_original_urls = []  # Better variable name for clarity

def process_save_markdown():
    count = 0
    files = [*Path("raw-data/Markdown-data").glob("*.md")]

    total_chunks = 0
    file_chunks = {}
    file_urls = {}  # Store original_urls for each file
    
    # First pass: extract content and count chunks
    for file_path in files:
        content, original_url = extract_text_from_markdown(file_path)
        chunks = get_chunks(content)
        file_chunks[file_path] = chunks
        file_urls[file_path] = original_url  # Store original_url for this file
        total_chunks += len(chunks)
        print(f"File: {file_path.name}, Chunks: {len(chunks)}")
    
    print(f"Total chunks created: {total_chunks}")
    
    # Second pass: process chunks with correct URLs
    with tqdm(total=total_chunks, desc="Processing Chunks") as pbar:
        for file_path, chunks in file_chunks.items():
            original_url = file_urls[file_path]  # Get correct original_url for this file
            for chunk in chunks:
                try:
                    embedding = get_embeddings(chunk, api_key=GEMINI_API_KEY)
                    all_chunks.append(chunk)
                    all_embeddings.append(embedding)
                    all_original_urls.append(original_url)  # Correct URL assignment
                    pbar.set_postfix({"file": file_path.name, "chunk": len(all_chunks)})
                except Exception as e:
                    print(f"Error processing chunk in {file_path}: {e}")
                finally:
                    pbar.update(1)
            break
    
    # Save once at the end
    # np.savez("markdown_embeddings.npz", 
    #          chunks=all_chunks, 
    #          embeddings=all_embeddings, 
    #          original_urls=all_original_urls)  # Updated field name

if __name__ == "__main__":
    process_save_markdown()
    print("Processing complete. Embeddings saved to 'markdown_embeddings.npz'.")

File: 1._Development_Tools.md, Chunks: 1
File: 2._Deployment_Tools.md, Chunks: 1
File: 3._Large_Language_Models.md, Chunks: 3
File: 4._Data_Sourcing.md, Chunks: 2
File: 5._Data_Preparation.md, Chunks: 1
File: 6._Data_Analysis.md, Chunks: 1
File: 7._Data_Visualization.md, Chunks: 1
File: Actor_Network_Visualization.md, Chunks: 1
File: AI_Code_Editors__GitHub_Copilot.md, Chunks: 1
File: AI_Terminal_Tools__llm.md, Chunks: 3
File: Authentication__Google_Auth.md, Chunks: 5
File: Base_64_Encoding.md, Chunks: 3
File: BBC_Weather_API_with_Python.md, Chunks: 4
File: Browser__DevTools.md, Chunks: 2
File: CI_CD__GitHub_Actions.md, Chunks: 4
File: Cleaning_Data_with_OpenRefine.md, Chunks: 1
File: Containers__Docker,_Podman.md, Chunks: 3
File: Convert_HTML_to_Markdown.md, Chunks: 11
File: Convert_PDFs_to_Markdown.md, Chunks: 10
File: Correlation_with_Excel.md, Chunks: 1
File: CORS.md, Chunks: 3
File: Crawling_with_the_CLI.md, Chunks: 6
File: CSS_Selectors.md, Chunks: 2
File: Database__SQLite.md, Ch

Processing Chunks:   0%|          | 1/321 [00:01<06:28,  1.21s/it, file=1._Development_Tools.md, chunk=1]

Processing complete. Embeddings saved to 'markdown_embeddings.npz'.


In [6]:
all_chunks

['Development Tools\n\n\nNOTE\n: The tools in this module are \nPRE-REQUISITES\n for the course. You would have used most of these before. If most of this is new to you, please take this course later.\n\n\nSome tools are fundamental to data science because they are industry standards and widely used by data science professionals. Mastering these tools will align you with current best practices and making you more adaptable in a fast-evolving industry.\n\n\nThe tools we cover here are not just popular, they’re the core technology behind most of today’s data science and software development.']

In [7]:
all_embeddings

[[-0.0030159312,
  0.008317691,
  -0.011245703,
  -0.045830954,
  0.018427646,
  0.027459772,
  0.011094433,
  -0.010273768,
  0.0050145304,
  -0.007550763,
  -0.021047011,
  -0.019075407,
  0.028204415,
  0.019369092,
  0.12633488,
  -0.0041779154,
  -0.0035645585,
  -0.020541403,
  0.019815855,
  -0.01316923,
  -0.004860248,
  0.020668473,
  -0.022239465,
  -0.027180897,
  0.02693248,
  -0.018179368,
  0.017592886,
  -0.004785976,
  0.047948945,
  0.0023677798,
  -0.01350658,
  -0.0064261886,
  0.0064037917,
  0.04743043,
  0.0054552895,
  0.020469079,
  -0.0055583706,
  -0.0038748803,
  0.020863885,
  0.0071318387,
  -0.018032731,
  -0.0023997265,
  -0.0005085406,
  -0.035326947,
  0.0054187346,
  0.021994716,
  -0.007703004,
  0.005296304,
  -0.01865234,
  0.0109934965,
  -0.011986173,
  -0.0057299994,
  -0.021746712,
  -0.164357,
  -0.0042222277,
  -0.001961825,
  -0.007287386,
  0.02198064,
  -0.0052574356,
  -0.008716809,
  -0.02057189,
  0.008677038,
  -0.051399294,
  0.0018100

In [8]:
all_original_urls

['https://tds.s-anand.net/#/development-tools?id=development-tools']